In [ ]:
### create segmentation mask from  rle ###

import os
import pandas as pd
import numpy as np
from PIL import Image

df = pd.read_csv('/data/train.csv')
column_names = list(df.columns)

img_path = '/data/train_images'
save_path = '/data/train_masks'


def create_mask(rle, w, h):
    mask = np.zeros(h*w, dtype=np.byte)
    for i in range(0, len(rle), 2):
        mask[rle[i]: rle[i] + rle[i+1]] = 1
    
    mask = np.reshape(mask, (h, w))
    return mask


for id, row in df.iterrows():
    fn = os.path.join(img_path, str(row['id']) + '.tiff')
    w, h = Image.open(fn).size

    rle = list(map(int, row['rle'].split(' ')))
    mask = create_mask(rle, w, h)

    pil_mask = Image.fromarray(mask)
    pil_mask.save(os.path.join(save_path, str(row['id']) + '.png'))

In [39]:
### create class index annotated txt file  ###

CLASS_LABEL = {
    'kidney': 0,
    'largeintestine': 1,
    'lung': 2,
    'prostate': 3,
    'spleen': 4
}

import os
import pandas as pd
import numpy as np
from PIL import Image

df = pd.read_csv('/data/train.csv')

with open('/data/train_cls.txt', 'w') as outf:
    with open('/data/train.txt', 'r') as inf:
        for id in inf.readlines():
            organ = df.loc[ df['id'] == int(id[:-1])]['organ'].to_string(index=False)
            outf.writelines(str(id[:-1]) + ' ' + str(CLASS_LABEL[organ]) + '\n')

with open('/data/val_cls.txt', 'w') as outf:
    with open('/data/val.txt', 'r') as inf:
        for id in inf.readlines():
            organ = df.loc[ df['id'] == int(id[:-1])]['organ'].to_string(index=False)
            outf.writelines(str(id[:-1]) + ' ' + str(CLASS_LABEL[organ]) + '\n')